# 🔧 Utilities

Some utilities for demos, used in other notebooks.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import asyncio
import json
import random
import traceback
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
from warnings import warn

import ipywidgets as W
import traitlets as T

import ipyforcegraph.behaviors as B
import ipyforcegraph.graphs as G
import ipyforcegraph.sources as S

In [ ]:
colors = [
    "rgba(166,206,227,1.0)",
    "rgba(31,120,180,1.0)",
    "rgba(178,223,138,1.0)",
    "rgba(51,160,44,1.0)",
    "rgba(251,154,153,1.0)",
    "rgba(227,26,28,1.0)",
    "rgba(253,191,111,1.0)",
    "rgba(255,127,0,1.0)",
    "rgba(202,178,214,1.0)",
    "rgba(106,61,154,1.0)",
    "rgba(255,255,153,1.0)",
    "rgba(177,89,40,1.0)",
]

In [ ]:
def make_a_demo(source=None, dataset="datasets/miserables.json", GraphClass=None):
    GraphClass = GraphClass or G.ForceGraph
    if source is None:
        data = json.loads(Path(dataset).read_text())
        source = S.DataFrameSource(**data)
    fg = GraphClass(source=source, layout=dict(min_height="500px", flex="1"))
    style = W.HTML(
        """<style>
        .jp-fg-demo{
            --jp-widgets-container-padding: 0.25em;
            --jp-widgets-inline-width: auto;
        } 
        .widget-box:empty{display:none;}
    </style>"""
    )
    description_ui = W.HTML(layout=dict(flex="0"))
    graph_ui = W.VBox()
    node_ui = W.VBox()
    link_ui = W.VBox()
    ui = W.VBox(
        [graph_ui, node_ui, link_ui], layout=dict(width="500px", overflow_y="scroll")
    )
    layout = dict(height="100%", max_height="100vh")
    fg_wrap = W.VBox([description_ui, fg], layout=dict(flex="1", **layout))
    box = W.HBox([style, fg_wrap, ui], layout=layout)
    box.add_class("jp-fg-demo")
    box.add_traits(
        behaviors=T.Dict(),
        node_ui=T.Dict(),
        link_ui=T.Dict(),
        graph_ui=T.Dict(),
        description=T.Unicode(),
    )

    def on_box_behaviors(change=None):
        new_behaviors = [b for b in box.behaviors.values() if b is not None]
        fg.behaviors = tuple(new_behaviors)

    box.observe(on_box_behaviors, ["behaviors"])

    T.dlink((box, "graph_ui"), (graph_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "node_ui"), (node_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "link_ui"), (link_ui, "children"), lambda d: tuple(d.values()))
    T.dlink((box, "description"), (description_ui, "value"), lambda d: d or "")
    return fg, box

In [ ]:
def make_an_rgba_picker(**colors):
    kw = {"min": 0, "layout": {"width": "100%"}}
    r, g, b = [
        W.IntSlider(colors.get(x, 0), description=x, max=255, **kw) for x in "rgb"
    ]
    a = W.FloatSlider(colors.get("a", 0), description="a", max=1, **kw)
    sliders = [r, g, b, a]
    box = W.VBox(sliders, layout=dict(width="100%"))
    box.add_traits(color=T.Unicode())

    def update(*args):
        return f"rgba({r.value}, {g.value}, {b.value}, {a.value})"

    [T.dlink((s, "value"), (box, "color"), update) for s in sliders]
    return box

In [ ]:
def make_a_collapsible_picker(title, children):
    ui_children = []
    ui = W.HBox(layout=dict(flex_wrap="wrap"))
    box = W.Accordion([ui], titles=[title])
    select = None
    if len(children) > 1:
        select = W.Dropdown(options=list(children), layout=dict(flex="0"))
        ui.children = [select]
        T.dlink((select, "value"), (ui, "children"), lambda x: [select, *children[x]])
        T.dlink((select, "value"), (box, "titles"), lambda x: (f"{title} ({x})",))
        box.titles = (f"{title} ({select.value})",)
    elif len(children) == 1:
        ui.children = tuple(list(children.values())[0])
    else:
        raise ValueError(f"unexpected number of children {children}")
    box._select = select
    return box

In [ ]:
def make_random_color_series(fg, trait, column_name):
    df = getattr(fg.source, trait)
    digits = "01234567abcdef"
    df[column_name] = [
        "#" + "".join([random.choice(digits) for j in range(6)]) for i in range(len(df))
    ]
    fg.source.send_state(trait)

In [ ]:
def make_link_dropdown_responsive(behavior, label, ui, box):
    box.behaviors = {**box.behaviors, label: None}

    def on_select(change: T.Bunch = None):
        new_behaviors = dict(box.behaviors.items())
        if ui._select.value == "off":
            new_behaviors[label] = None
        else:
            new_behaviors[label] = behavior
        box.behaviors = new_behaviors

    ui._select.observe(on_select, T.All)
    on_select()

In [ ]:
def make_link_behavior_with_ui(WidgetClass, label, column_name, is_color=False):
    def add_behavior(fg, box, column_name=column_name):
        behavior = WidgetClass(column_name=column_name)
        if is_color:
            if column_name not in fg.source.links:
                make_random_color_series(fg, "links", column_name)
        ui_column_name = W.Dropdown(
            options=list(fg.source.links.columns), value=column_name
        )
        subscribe_to_columns(fg, "links", ui_column_name, "options")
        ui_template = W.Textarea()
        ui_template_enabled = W.Checkbox(description="enabled?")
        T.link((ui_column_name, "value"), (behavior, "column_name"))
        T.dlink(
            (ui_template, "value"),
            (behavior, "template"),
            lambda x: x if ui_template_enabled.value else "",
        )

        ui = make_a_collapsible_picker(
            label,
            {
                "off": [],
                "column": [ui_column_name],
                "template": [W.VBox([ui_template_enabled, ui_template])],
            },
        )

        box.link_ui = {**box.link_ui, label: ui}

        make_link_dropdown_responsive(behavior, label, ui, box)

        return fg, box

    return add_behavior

In [ ]:
def subscribe_to_columns(fg, elements, ui, trait, tx=None):
    tx = tx or list

    def on_source_change(change):
        setattr(ui, trait, tx(getattr(fg.source, elements).columns))

    fg.observe(on_source_change, ["source"])

## Forces

In [ ]:
def ensure_graph_forces(fg, box):
    if "graph_forces" not in box.behaviors:
        box.behaviors = {**box.behaviors, "graph_forces": B.GraphForces()}

    return box.behaviors["graph_forces"]

In [ ]:
def make_a_force_picker(title, children):
    ui_children = []
    ui = W.HBox(layout=dict(flex_wrap="wrap"))
    box = W.Accordion([ui], titles=[title])
    select = None
    if len(children) > 1:
        options = [(key, value) for key, value in children.items()]
        select = W.Dropdown(options=options, layout=dict(flex="0"))
        ui.children = [select]
        T.dlink((select, "value"), (ui, "children"), lambda x: [select, *select.value])
        T.dlink(
            (select, "value"),
            (box, "titles"),
            lambda x: (f"{title} ({[n for n, v in select.options if v is x][0]})",),
        )
        box.titles = (f"{title} ({select.value})",)
    elif len(children) == 1:
        ui.children = tuple(list(children.values())[0])
    else:
        raise ValueError(f"unexpected number of children {children}")
    box._select = select
    return box

In [ ]:
def ensure_graph_force_ui(fg, box, children: dict):
    if "forces" not in box.graph_ui:
        ui = make_a_force_picker("Forces", {"link": [], "charge": [], "center": []})
        box.graph_ui = {**box.graph_ui, "forces": ui}

    ui = box.graph_ui["forces"]
    existing = {key: child for key, child in ui._select.options}
    for key, child in children.items():
        existing[key] = child

    ui._select.options = [(key, value) for key, value in existing.items()]
    return ui, existing

## Tests

In [ ]:
def wait_for_change(widget: W.Widget, value: Any, timeout=10) -> asyncio.Future:
    """Initial pattern from
    https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Asynchronous.html
    """

    future: asyncio.Future = asyncio.Future()

    def getvalue(change: T.Bunch) -> None:
        """make the new value available"""
        future.set_result(change.new)

    def unobserve(f: Any) -> None:
        """unobserves the `getvalue` callback"""
        widget.unobserve(getvalue, value)

    future.add_done_callback(unobserve)

    widget.observe(getvalue, value)

    return asyncio.wait_for(future, timeout)

In [ ]:
async def run_tests(fg, box, tests: list, timeout=30):
    result = ""
    errors = []
    for test in tests:
        box.description = test.__doc__

        try:
            await asyncio.wait_for(test(fg, box), timeout)
            box.description = f"✅ {test.__doc__}"
            result += "✅"
        except Exception as err:
            box.description = f"💥 {test.__doc__}"
            errors += [str(err), traceback.format_exc()]
            result += "💥"
    box.description = f"{result} Tests Completed"
    if errors:
        nl = "\n"
        box.description += f"""<pre>{nl.join(errors)}</pre>"""

# UI Generator

> These classes are intended to facilitate demonstrating the functionality in
> `ipyforcegraph`, they are **not** intended to be a standard means of making a UI for
> `ForceGraph`s.

> For more information on an automatic UI generator for controlling `Behavior` refer to
> [UI for Behaviors](https://github.com/jupyrdf/ipyforcegraph/issues/61)

In [ ]:
DEFAULT_LAYOUT = {"width": "auto"}


@W.register
class NunjucksUI(W.VBox):
    """A UI for specifying ``Behavior`` attributes using Nunjucks."""

    PLACEHOLDER = """`Nunjucks` take the form of nunjucks templates, this allows
for calculating dynamic values on the client. One can use:

- `node`
  - this will have all of the named columns available to it
- `link`
  - `source` and `target` as realized nodes
- `graphData`
  - `nodes`
  - `links`

The syntax is intentionally very similar to jinja2, and a number of extra
template functions are provided.

With these, and basic template tools, one can generate all kinds of interesting effects.
For the example data above, try these color templates:

- color by group, e.g.,
  {{ ["red", "yellow", "blue", "orange", "purple", "magenta"][node.group] }}

- color by out-degree, e.g.,
  {% set n = 0 %}
  {% for link in graphData.links %}
    {% if link.source.id == node.id %}{% set n = n + 1 %}{% endif %}
  {% endfor %}
  {% set c = 256 * (7-n) / 7 %}
  rgb({{ c }},0,0)
"""

    ROWS = 10

    value: str = T.Unicode("").tag()
    active: W.ToggleButton = T.Instance(
        W.ToggleButton,
        kw=dict(
            description="Active",
            layout=DEFAULT_LAYOUT,
            value=True,
        ),
    ).tag()
    textarea: W.Textarea = T.Instance(
        W.Textarea,
        kw=dict(
            placeholder=PLACEHOLDER,
            layout=DEFAULT_LAYOUT,
            rows=ROWS,
        ),
    ).tag()

    def _update_value(self, _: Optional[T.Bunch] = None) -> None:
        """Update the overall value based on the state of the textarea and activation control."""
        if self.active.value and self.textarea.value:
            self.value = self.textarea.value
        else:
            self.value = ""
        # self.value = self.textarea.value if self.active.value else None
        self.textarea.disabled = not self.active.value

    def __init__(self, *args: str, **kwargs: str):
        super().__init__(*args, **kwargs)
        for widget in (self.active, self.textarea):
            widget.observe(self._update_value, "value")

        self.children: tuple = tuple(self.children)
        if not self.children:
            self.children = (self.textarea, self.active)
        self._update_value()


@W.register
class BehaviorAttributeUI(W.Accordion):
    """A set of controls for setting the value of a Behavior Attribute."""

    BASE_TRAIT_NAMES = tuple(B.Behavior.class_traits())

    WIDGET_BY_TRAIT = {
        T.Bool: W.Checkbox,
        T.Float: W.FloatText,
        T.Int: W.IntText,
        T.Unicode: W.Text,
        B.Column: W.Dropdown,
        B.Nunjucks: NunjucksUI,
    }

    attribute_name: str = T.Unicode().tag()

    value: Optional[Union[T.TraitType, B.DynamicValue]] = T.Union(
        trait_types=[
            T.Unicode(),
            T.Float(),
            T.Int(),
            T.Instance(B.DynamicValue),
            T.Bool(),
        ],
        allow_none=True,
    ).tag()

    @T.observe("selected_index")
    def _update_value(self, *_: T.Bunch) -> None:
        if self.selected_index is None:
            return
        active_child = self.children[self.selected_index]
        value = active_child.value
        if value in (None, False, [], {}, set()):
            self.value = None
            return
        if hasattr(active_child, "options") and value in active_child.options:
            self.value = B.Column(value)
            return
        if isinstance(active_child, NunjucksUI):
            self.value = B.Nunjucks(value)
            return
        self.value = value

    @T.validate("children")
    def _validate_children(self, proposal: T.Bunch) -> List[W.DOMWidget]:
        children = proposal.value or []
        for child in children:
            child.observe(self._update_value, "value")
        return children

    @classmethod
    def _get_trait_classes(
        cls, trait: T.TraitType, classes: Optional[List[Any]] = None
    ) -> List[Any]:
        """Recursive method to find all the trait classes allowed."""
        classes = classes or []
        if isinstance(trait, T.Instance):
            return classes + [trait.klass]

        if isinstance(trait, T.Union):
            for trait_type in trait.trait_types:
                classes += cls._get_trait_classes(trait_type)
            return classes

        return classes + [trait.__class__]

    @classmethod
    def make_behavior_controls(
        cls, behavior: B.Behavior, options: Tuple[str, ...]
    ) -> W.Accordion:
        """Make UI controls for a given behavior."""
        behavior_trait_classes = {
            name: cls._get_trait_classes(trait)
            for name, trait in behavior.traits().items()
            if name not in cls.BASE_TRAIT_NAMES
        }

        widgets = {
            label: {
                f"From {t.__name__}": cls.WIDGET_BY_TRAIT[t]
                for t in traits
                if t in cls.WIDGET_BY_TRAIT
            }
            for label, traits in behavior_trait_classes.items()
        }

        trait_controls, trait_labels = [], []
        for label, controls in widgets.items():
            titles, children = zip(*controls.items())
            additional_kwargs = [
                dict(options=options) if hasattr(child, "options") else {}
                for child in children
            ]
            children = [
                # This is where the UI controls get instantiated
                child(
                    layout=DEFAULT_LAYOUT,
                    **kwargs,
                )
                for child, kwargs in zip(children, additional_kwargs)
            ]
            attribute_ui = cls(
                attribute_name=label,
                children=children,
                titles=titles,
            )
            T.dlink((attribute_ui, "value"), (behavior, label))
            trait_controls.append(attribute_ui)
            trait_labels.append(label.title())
        return W.Accordion(
            children=trait_controls,
            titles=trait_labels,
        )


@W.register
class GraphBehaviorsUI(W.Accordion):
    """An auto-generated UI for a ForceGraph Behavior."""

    graph: G.ForceGraph = T.Instance(G.ForceGraph).tag()
    show_warnings: bool = T.Bool(False).tag()

    IGNORED_COLUMNS: Dict[str, List[str]] = {
        "nodes": [],
        "links": ["source", "target"],
    }

    def _make_ui_for_behavior(self, behavior: B.Behavior) -> W.Accordion:
        """Make the ui for a single behavior"""
        behavior_name = behavior.__class__.__name__.lower()
        if "node" in behavior_name:
            context = "nodes"
        elif "link" in behavior_name:
            context = "nodes"
        else:
            raise NotImplementedError(
                f"Cannot determine if '{behavior.__class__.__name__}' operates on `nodes` or `links`."
            )
        ignored_columns: List[str] = self.IGNORED_COLUMNS[context]

        return BehaviorAttributeUI.make_behavior_controls(
            behavior,
            options=tuple(
                sorted(
                    [
                        column
                        for column in getattr(self.graph.source, context).columns
                        if column not in ignored_columns
                    ]
                )
            ),
        )

    def _on_new_behaviors(self, *_: T.Bunch) -> None:
        """Run when graph receives new behaviors."""
        children, titles = [], []
        for behavior in self.graph.behaviors:
            behavior_ui = self._cached_widgets.get(behavior)
            title = self._cached_titles.get(behavior)
            if behavior_ui is None:
                title = behavior.__class__.__name__
                try:
                    behavior_ui = self._make_ui_for_behavior(behavior)
                except (NotImplementedError, T.TraitError) as exc:
                    if self.show_warnings:
                        warn(str(exc))
                    continue
                if len(behavior_ui.children) == 1:
                    # Remove unnecessary nesting for single attribute behaviors
                    child = behavior_ui.children[0]
                    title += f" ({behavior_ui.titles[0]})"
                    behavior_ui.children = child.children
                    behavior_ui.titles = child.titles
                self._cached_widgets[behavior] = behavior_ui
                self._cached_titles[behavior] = title
            children += [behavior_ui]
            titles += [title]
        self.children = children
        self.titles = titles

    @T.observe("graph")
    def _on_new_graph(self, change: T.Bunch) -> None:
        if isinstance(change.old, G.ForceGraph):
            change.old.unobserve(self._on_new_behaviors)
        if isinstance(change.new, G.ForceGraph):
            change.new.observe(self._on_new_behaviors, "behaviors")
            self._on_new_behaviors()

    def __init__(self, *args: G.ForceGraph, **kwargs: G.ForceGraph):
        self._cached_widgets: Dict[B.Behavior, W.DOMWidget] = {}
        self._cached_titles: Dict[B.Behavior, str] = {}

        super().__init__(*args, **kwargs)
        self._on_new_graph(T.Bunch(old=None, new=self.graph))